Import all the necessary libraries

In [101]:
import shapely
from shapely.geometry import box,Polygon
from shapely.wkt import loads
import csv
import folium
import geopandas
from fiona.crs import from_epsg

Step 1 : Read the shape file which is storted locally to derive the geometry

In [ ]:
##read a wkt file which is in format lon,lat
data=[]
with open('myfilepath.wkt',mode='r') as wktfile:  ##add the path where the file is stored locally
    reader = csv.reader(wktfile, delimiter='\t')
    for i,row in enumerate(reader):
        data.append(row[0])
       
for wkt in data:
    geometry=loads(wkt)

#print(geometry)

Step 2 :  Transform the geometry to GeoDataFrame

In [ ]:
#transfrom the polygon to GeoDataFrame
gdf = geopandas.GeoSeries(geometry)
gdf.crs = 'EPSG:4326'

gdf_4326 = geopandas.GeoDataFrame(gdf)
gdf_4326 = gdf.to_crs('EPSG:4326')

# Ensure to remove islands
gdf_4326 = gdf_4326.apply(lambda g: Polygon(g.exterior))

# Create single polygon geometries
gdf_4326 = gdf_4326.explode()


Step 3: Divide the box based on (ABS((MAXLAT - MINLAT) + (MAXLON - MINLON)) <= 2)

In [ ]:

#A function which divides the initial geometry to boxes 

def divide_bounding_box(geometry,delta=2):
    result = []
    min_lon, min_lat, max_lon, max_lat = geometry.bounds
    queue = [(min_lon, min_lat, max_lon, max_lat)]

    while queue:
        min_lon, min_lat, max_lon, max_lat = queue.pop(0)
        
        if abs((max_lat - min_lat) + (max_lon - min_lon)) <= delta:
            
            segment_box = box(min_lon, min_lat, max_lon, max_lat)
            
            if segment_box.intersects(geometry):
                result.append((min_lon, min_lat, max_lon, max_lat))
                
        else:
            mid_lat = (min_lat + max_lat) / 2
            mid_lon = (min_lon + max_lon) / 2

            queue.append((min_lon, min_lat, mid_lon, mid_lat))
            queue.append((mid_lon, min_lat, max_lon, mid_lat))
            queue.append((min_lon, mid_lat, mid_lon, max_lat))
            queue.append((mid_lon, mid_lat, max_lon, max_lat))

    
    return result

# Get the divided boxes
divided_boxes = divide_bounding_box(geometry,2)

#print(divided_boxes)


Step 4: Visualise both the initial polygon and the boxes

In [ ]:
## visualise the initial geometry and the boxes
map = folium.Map([48,10], zoom_start=5, tiles='OpenStreetMap')

bbox_list=[]
for b in divided_boxes:
    min_lon, min_lat, max_lon, max_lat = b
    bbox = box(min_lon, min_lat, max_lon, max_lat)
    bbox_list.append((bbox))
    folium.GeoJson(bbox).add_to(map)
    

folium.GeoJson(geometry,style_function=lambda x: {'fillColor': 'white','color': 'orange'}).add_to(map) 

map

Step 5: Get the bounding box coordinates in a csv file

In [ ]:
# extract the Lat,Lon of the bounding boxes for each polygon 

s = geopandas.GeoSeries(bbox_list, crs=from_epsg(4326))
final=s.get_coordinates(index_parts=True)


#Rename the columns and the indexes
final=final.rename(columns={"x": "Lon", "y": "Lat"})
final=final.rename_axis(index=["Polygon_id", "Tuple_id"])
print(final)

#save them to csv file
final.to_csv('myfilepath.csv') ##add the path where you would like to store the file
